In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
# 너무 긴 예제를 사용하면 비용이 많이 들기때문에 어느 정도 길이를 갖고 있는 examples을 골라서 prompt에 허용할 것인지 정해야함
# 모델에 알맞은 길이의 예제를 제공하는 것이 중요
# 이런 목적을 달성하기 위해 LengthBasedExampleSelector를 import했음

# 이외에도 유저의 로그인 여부 또는 유저가 사용하는 언어에 따라 얼마나 examples들을 허용할지 등을 선택하도록 자시만의 example_stelector를 만들 수도 있음 


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# 모델이 나에게 어떤형식으로 대답해주면 좋겠는지 예제를 작성해서 알려줌
examples = [
    {
        "question": "What do you know about France?",
        "answer":"""
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },{
        "question": "What do you know about Italy?",
        "answer":"""
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },{
        "question": "What do you know about Greece?",
        "answer":"""
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    }
]

# 예제의 형식을 지정
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180, #예제의 길이를 제한(한 예제의 글자 수)
)

# FewShotPromptTemplate을 이용하여 예제들과 유사한 형식의 template을 생성
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Humman: What do you know about {country}?", # 예제들의 마지막에 이어붙여질 질문. 이 질문에 대해 AI는 앞의 예제와 똑같은 형식으로 답변을 할 것임
    input_variables=["country"]
)

prompt.format(country="Brazil")

'\n    Human: What do you know about France?\n    AI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Italy?\n    AI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\nHumman: What do you know about Brazil?'